## imports

In [1]:
import os
import sys
sys.path.append("../")

import math

import pandas as pd
import numpy as np
import datetime as dt
import re

import pymc3 as pm

import altair as alt
alt.data_transformers.disable_max_rows()

from src.preprocess.preprocess import load_data, split_last
from src.plot.altair import plot_decompose, plot_all
from src.forecast.forecast import determine_estimates_minmax, forecast

In [2]:
boxid = [
    "ESD.000088-1",
    "063.623-1",
    "VRY.CHOPS-1",
    "HVT.111153-1",
    "TTR.251049-1",
    "BGL.CROLA-1",
][3]

## load data

In [3]:
boxid = "063.691-1"

In [4]:
%%time
df_data, df_meta = load_data(boxid=boxid)
plot_all(df_data=df_data)

[Fri, 19 Nov 2021 10:58:13] INFO [snowflake.py.read_meta] reading meta preprocess
[Fri, 19 Nov 2021 10:58:13] INFO [snowflake.py.read_meta] for boxids: 063.691-1
[Fri, 19 Nov 2021 10:58:17] INFO [snowflake.py.read_week_extremes] reading extremes table
[Fri, 19 Nov 2021 10:58:21] INFO [preprocess.py.too_short] checking number of preprocess points (<=104)


INFO:SPARK:checking number of preprocess points (<=104)


[Fri, 19 Nov 2021 10:58:21] INFO [preprocess.py.too_small] checking absolute values (<0.5)


INFO:SPARK:checking absolute values (<0.5)


[Fri, 19 Nov 2021 10:58:21] INFO [preprocess.py.remove_leading_idling] removing leading low values (<0.01)


INFO:SPARK:removing leading low values (<0.01)


[Fri, 19 Nov 2021 10:58:21] INFO [preprocess.py.too_short] checking number of preprocess points (<=104)


INFO:SPARK:checking number of preprocess points (<=104)


CPU times: user 796 ms, sys: 125 ms, total: 921 ms
Wall time: 7.4 s


alt.LayerChart(...)

## split in train + test

In [5]:
df_train, df_test = split_last(df_data.copy())

## estimate parameters + forecast

In [6]:
%%time
df_estimates = determine_estimates_minmax(df_train)
df_total=pd.concat([df_data, df_estimates], axis=0)

[Fri, 19 Nov 2021 10:58:21] INFO [forecast.py.determine_estimates_minmax] forecast for weekly min


INFO:SPARK:forecast for weekly min


[Fri, 19 Nov 2021 10:58:21] INFO [forecast.py.determine_estimates] add forecast horizon


INFO:SPARK:add forecast horizon


[Fri, 19 Nov 2021 10:58:21] INFO [forecast.py.determine_estimates] scale data


...

[Fri, 19 Nov 2021 11:01:27] INFO [format.py.format_model_estimates] calculating bands for variable: yearly


INFO:SPARK:calculating bands for variable: yearly


[Fri, 19 Nov 2021 11:01:27] INFO [format.py.format_model_estimates] calculating bands for variable: Σ


INFO:SPARK:calculating bands for variable: Σ


CPU times: user 20.1 s, sys: 16.7 s, total: 36.8 s
Wall time: 3min 6s


## plot results

In [10]:
plot_all(df_data=df_total, df_meta=df_meta).properties(width=800)

alt.LayerChart(...)

In [9]:
plot_decompose(df=df_total.query("extreme == 'min'"))

SchemaValidationError: Invalid specification

        altair.vegalite.v4.api.VConcatChart, validating 'additionalProperties'

        Additional properties are not allowed ('width' was unexpected)
        

alt.VConcatChart(...)

In [ ]:
df_total